# This case study consists of 1000 marks and the solution file is expected to be submitted in the upcoming session for evaluation.

# Combining Datasets: Merge and Join

One essential feature offered by Pandas is its high-performance, in-memory join and merge operations.
If you have ever worked with databases, you should be familiar with this type of data interaction.
The main interface for this is the ``pd.merge`` function, and we'll see few examples of how this can work in practice.

For convenience, we will start by redefining the ``display()`` functionality from the previous section:

In [2]:
import pandas as pd
import numpy as np


## Specification of the Merge Key

We've already seen the default behavior of ``pd.merge()``: it looks for one or more matching column names between the two inputs, and uses this as the key.
However, often the column names will not match so nicely, and ``pd.merge()`` provides a variety of options for handling this.

## Example: US States Data

Merge and join operations come up most often when combining data from different sources.
Here we will consider an example of some data about US states and their populations.
The data files can be found at http://github.com/jakevdp/data-USstates/:

In [12]:
# Following are shell commands to download the data
! mkdir -p data
!cd data ; curl -O  https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv ; cd ..
!cd data ; curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv ; cd ..
!cd data ; curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv ; cd ..

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57935  100 57935    0     0  78182      0 --:--:-- --:--:-- --:--:-- 78184
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   835  100   835    0     0   2553      0 --:--:-- --:--:-- --:--:--  2553
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   872  100   872    0     0   2720      0 --:--:-- --:--:-- --:--:--  2725


In [13]:
!ls

03.06-Concat-And-Append.ipynb  data		      data_exploration.ipynb
03.07-Merge-and-Join.ipynb     Data Cleaning 1.ipynb


Let's take a look at the three datasets, using the Pandas ``read_csv()`` function:

In [25]:
pop = pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')

In [13]:
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [14]:
areas.head()

,state area,(sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [15]:
abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


Given this information, say we want to compute a relatively straightforward result: rank US states and territories by their 2010 population density.
We clearly have the data here to find this result, but we'll have to combine the datasets to find the result.

We'll start with a many-to-one merge that will give us the full state name within the population ``DataFrame``.
We want to merge based on the ``state/region``  column of ``pop``, and the ``abbreviation`` column of ``abbrevs``.
We'll use ``how='outer'`` to make sure no data is thrown away due to mismatched labels.

## Merge the 2 dataframes, pop and abbrevs such that all rows in both tables are included. Drop any duplicate column in the merged dataframe.

In [16]:
merged = pop.merge(abbrevs,left_on='state/region', right_on='abbreviation', how='outer')
merged.head(2)
merged.shape

(2544, 6)

Let's double-check whether there were any mismatches here, which we can do by looking for rows with nulls:

In [17]:
merged =merged.drop('abbreviation',axis=1)
merged.shape

(2544, 5)

## Look for any rows with null values in the new dataframe

In [18]:
merged.isnull().sum()

state/region     0
ages             0
year             0
population      20
state           96
dtype: int64

## Print out the rows in the merged dataframe that has null values for population.

In [19]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

It appears that all the null population values are from Puerto Rico prior to the year 2000; this is likely due to this data not being available from the original source.

More importantly, we see also that some of the new ``state`` entries are also null, which means that there was no corresponding entry in the ``abbrevs`` key!
Let's figure out which regions lack this match:

In [20]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

We can quickly infer the issue: our population data includes entries for Puerto Rico (PR) and the United States as a whole (USA), while these entries do not appear in the state abbreviation key.
We can fix these quickly by filling in appropriate entries:

In [21]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

No more nulls in the ``state`` column: we're all set!

Now we can merge the result with the area data using a similar procedure.
Examining our results, we will want to join on the ``state`` column in both:

In [26]:
final = pd.merge(merged, areas, on='state', how='left')
final.head(2)

,state/region,ages,year,population,state,area(sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0


In [27]:
final = pd.merge(merged, areas, on='state', how='outer')
final.isnull().sum()

state/region     0
ages             0
year             0
population      20
state            0
area(sq. mi)    48
dtype: int64

## Repeat the process of checking for nulls in the area field for each state in the merged dataframe and deal with them appropriately.

In [28]:
final[final['state'].isnull()]['state/region'].unique()

array([], dtype=object)

In [29]:
final.loc[final['state/region'] =='PR', 'state'] = 'Puerto Rico'
final.loc[final['state/region'] == 'USA', 'state'] = 'United States'
final['state'].isnull().sum()

0

## Select the portion of the data corresponding with the year 2000, and the total population.

In [30]:
final[final['year'] == 2000]

,state/region,ages,year,population,state,area(sq. mi)
28,AL,total,2000,4452173.0,Alabama,52423.0
29,AL,under18,2000,1122273.0,Alabama,52423.0
68,AK,total,2000,627963.0,Alaska,656425.0
69,AK,under18,2000,190615.0,Alaska,656425.0
124,AZ,total,2000,5160586.0,Arizona,114006.0
125,AZ,under18,2000,1373414.0,Arizona,114006.0
162,AR,total,2000,2678588.0,Arkansas,53182.0
163,AR,under18,2000,680378.0,Arkansas,53182.0
220,CA,total,2000,33987977.0,California,163707.0
221,CA,under18,2000,9267089.0,California,163707.0


## Re-index the data on state and find population density and display by density in ascending order

The result is a ranking of US states plus Washington, DC, and Puerto Rico in order of their 2010 population density, in residents per square mile.
We can see that by far the densest region in this dataset is Washington, DC (i.e., the District of Columbia); among states, the densest is New Jersey.

We can also check the end of the list:

We see that the least dense state, by far, is Alaska, averaging slightly over one resident per square mile.

This type of messy data merging is a common task when trying to answer questions using real-world data sources.
I hope that this example has given you an idea of the ways you can combine tools we've covered in order to gain insight from your data!

In [31]:
final.reindex(columns=['state']).head(2)

,state
0,Alabama
1,Alabama
